In [1]:
from datetime import date
import calendar as c
import pandas as pd
import numpy as np

#### Para testear las funciones, levanto los dataframes originales

In [2]:
df_pageviews = pd.read_csv('data/pageviews.csv', parse_dates=['FEC_EVENT'])
df_adoption = pd.read_csv("./data/conversiones.csv")
year = 2018
mes_snapshot = 9

def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

universo = universo_train(df_pageviews)

In [3]:
def _sum_campo_user(df_pageviews, campo):
    temp = pd.crosstab(df_pageviews.USER_ID, df_pageviews[campo]) 
    temp.columns = [campo + "_" + str(v) for v in temp.columns]
    return temp.reset_index()

In [4]:
def _sum_total_user(df_pageviews, campo):
    return df_pageviews.groupby("USER_ID")[campo].sum()

In [5]:
def _ratio_campo_df(df_numerador, df_denominador, name):
    temp = df_numerador.div(df_denominador, axis=0)
    temp.columns = [name + "_" + str(v) for v in temp.columns]
    return temp

In [6]:
def _ratio_campo_series(df_numerador, df_denominador, name):
    temp = df_numerador.div(df_denominador, axis=0).to_frame()
    temp.columns = [name + "_" + str(v) for v in temp.columns]
    return temp

In [7]:
def _universo_last_adoption_month(universo, df_adoption):
    df_last_adoption = df_adoption.groupby('USER_ID')[('mes','anio')].max().reset_index()
    return df_last_adoption

In [8]:
def _get_valid_pageviews_last_adoption(universo, df_pageviews, df_adoption):
    df_last_adoption = _universo_last_adoption_month(universo, df_adoption)
    df_last_adoption_page = pd.merge(df_last_adoption, df_pageviews, how='left', left_on=['USER_ID'], right_on=['USER_ID'])

    df_last_adoption_page['last_adoption_month'] = pd.to_datetime(dict(year=df_last_adoption_page.anio, month=df_last_adoption_page.mes, day=1))
    df_last_adoption_page['valid_data_from'] = df_last_adoption_page.last_adoption_month + pd.DateOffset(months=-1)
    df_last_adoption_page['valid_data_to'] = df_last_adoption_page.last_adoption_month
    
    df_last_adoption_page = df_last_adoption_page[df_last_adoption_page['FEC_EVENT'] >= df_last_adoption_page['valid_data_from']]
    df_last_adoption_page = df_last_adoption_page[df_last_adoption_page['FEC_EVENT'] < df_last_adoption_page['valid_data_to']]
    df_last_adoption_page = pd.merge(universo, df_last_adoption_page, how='left', left_on=['user_id'], right_on=['USER_ID'])
    
    return df_last_adoption_page

In [9]:
def _get_valid_pageviews_last_month(universo, df_pageviews, mes_snapshot):
    df_last_month = pd.merge(universo, df_pageviews, how='left', left_on=['user_id'], right_on=['USER_ID'])
    
    df_last_month['valid_data_to'] = pd.to_datetime(dict(year=year, month=df_last_month.snapshot_mes, day=1))
    df_last_month['valid_data_from'] = df_last_month.valid_data_to + pd.DateOffset(months=-1)
    
    df_last_month = df_last_month[df_last_month['FEC_EVENT'] >= df_last_month['valid_data_from']]
    df_last_month = df_last_month[df_last_month['FEC_EVENT'] < df_last_month['valid_data_to']]
    df_last_month = pd.merge(universo, df_last_month, how='left', left_on=['user_id'], right_on=['USER_ID'])
    return df_last_month

In [10]:
def _all_pageview_last_adoption_and_last_month(universo, df_pageviews, df_adoption, mes_snapshot, campo):
    pageviews_last_adoption = _get_valid_pageviews_last_adoption(universo, df_pageviews, df_adoption)
    pageviews_last_adoption_sum = _sum_campo_user(pageviews_last_adoption, campo)
    pageviews_last_adoption_sum_tot = _sum_total_user(pageviews_last_adoption, campo)
    
    pageviews_last_month = _get_valid_pageviews_last_month(universo, df_pageviews, mes_snapshot)
    pageviews_last_month_sum = _sum_campo_user(pageviews_last_month, campo)
    pageviews_last_month_sum_tot = _sum_total_user(pageviews_last_month, campo)
    
    return pageviews_last_adoption_sum, pageviews_last_adoption_sum_tot, pageviews_last_month_sum, pageviews_last_adoption_sum_tot

In [28]:
def _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo):
    pageviews_last_adoption_sum, pageviews_last_adoption_sum_tot, pageviews_last_month_sum, pageviews_last_adoption_sum_tot = _all_pageview_last_adoption_and_last_month(universo, df_pageviews, df_adoption, mes_snapshot, campo)

    ratio_preferencia = _ratio_campo_df(pageviews_last_adoption_sum, pageviews_last_month_sum, 'ratio_adoption_preferencia')
    ratio_interes = _ratio_campo_series(pageviews_last_adoption_sum_tot, pageviews_last_adoption_sum_tot, 'ratio_adoption_interes')

    #df_union = pd.merge(universo, ratio_preferencia, how='left', left_on=['user_id'], right_on=['USER_ID'])
    #df_union = pd.merge(df_union, ratio_interes, how='left', left_on=['user_id'], right_on=['user_id'])

    return ratio_preferencia #df_union

,user_id,snapshot_mes
0,0,9
1,1,9
2,2,9
3,3,9
4,4,9
5,5,9
6,6,9
7,7,9
8,8,9
9,9,9


In [27]:
campo = 'PAGE'
pepe = _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)
pepe.head(10)

In [22]:
def ratio_page_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'PAGE'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_ccategory_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'CONTENT_CATEGORY'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_cctop_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'CONTENT_CATEGORY_TOP'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_ccbottom_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'CONTENT_CATEGORY_BOTTOM'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_site_id_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'SITE_ID'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_mobile_device_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'IS_MOBILE_DEVICE'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)

In [ ]:
def ratio_connection_speed_all(universo, df_pageviews, df_adoption, mes_snapshot):
    campo = 'CONNECTION_SPEED'
    return _ratio_campo_all(universo, df_pageviews, df_adoption, mes_snapshot, campo)